In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from spmf import Spmf
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import ast
pd.options.plotting.backend = "plotly"
import plotly.express as px
# from chart_studio import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.subplots as sp
pd.options.display.max_colwidth = None
pd.set_option('display.max_rows', 70)
from pandas.plotting import table 
import nltk
import seaborn as sns
%matplotlib inline
import random
import matplotlib.gridspec as grid
import matplotlib

In [2]:
data=pd.read_csv("sorted_coded_session_293s_merged.csv")
data['start']=data.groupby('user_id').session.shift(1)!=data.session
data['end']=data.groupby('user_id').session.shift(-1)!=data.session

In [3]:
data_individual_app_count = data.groupby(['user_id','package_name'])\
                            .agg({'session':'size'})\
                            .rename(columns={'session':'app_count'})

In [9]:
data_individual_app_count

app_count
user_id package_name                          
0       air.ca.udesignit.kitchen             4
        bbc.mobile.weather                 102
        bg.abv.andro.emailapp              510
        com.acobot.en                        2
        com.adobe.reader                   159
...                                        ...
9999    com.whatsapp                      2182
        uk.co.ee.myee                       16
        uk.co.nationalrail.google           11
        uk.co.santander.santanderUK         44
        uk.co.tsb.mobilebank                 3

[924071 rows x 1 columns]

In [15]:
individual_start_count=data.groupby(['user_id']).start.sum().reset_index()#,columns=['app_count'],index=['user_id','start'])
individual_start_count['package_name']='start'
individual_start_count=individual_start_count.rename(columns={'start':'app_count'})

In [28]:
data_individual_app_count=data_individual_app_count.append(individual_start_count.set_index(['user_id','package_name']))

In [4]:
data['next_app'] = data.groupby(['user_id','session']).package_name.shift(-1).fillna("end")

In [8]:
data_individual_next_app_count=data.groupby(['user_id','package_name','next_app'])\
                            .agg({'session':'size'})\
                            .rename(columns = {'session':'next_count'})

In [34]:
df_individual_numerator = data_individual_next_app_count.reset_index()
data_individual_app_count=data_individual_app_count.reset_index()

In [35]:
df_transition = df_individual_numerator.merge(data_individual_app_count, on=['user_id','package_name'])
df_transition['probability'] =df_transition.next_count/df_transition.app_count*100
df_transition.drop(['next_count', 'app_count'], axis=1, inplace=True)

In [43]:
df_transition

,user_id,package_name,next_app,probability
0,0,air.ca.udesignit.kitchen,com.facebook.katana,25.000000
1,0,air.ca.udesignit.kitchen,com.google.android.youtube,25.000000
2,0,air.ca.udesignit.kitchen,com.sonyericsson.android.socialphonebook,25.000000
3,0,air.ca.udesignit.kitchen,com.viber.voip,25.000000
4,0,bbc.mobile.weather,bg.abv.andro.emailapp,2.941176
...,...,...,...,...
10771624,9999,uk.co.santander.santanderUK,com.whatsapp,4.545455
10771625,9999,uk.co.santander.santanderUK,end,25.000000
10771626,9999,uk.co.tsb.mobilebank,com.americanexpress.android.acctsvcs.uk,33.333333
10771627,9999,uk.co.tsb.mobilebank,com.android.incallui,33.333333


The full information about all transitions are saved in df_transition, to access the transition matrix for a user, the following code can be used.

In [45]:
df_transition[df_transition.user_id==0].pivot(index=['user_id','package_name'], columns="next_app", values="probability").fillna(0)

next_app                                          air.ca.udesignit.kitchen  \
user_id package_name                                                         
0       air.ca.udesignit.kitchen                                  0.000000   
        bbc.mobile.weather                                        0.000000   
        bg.abv.andro.emailapp                                     0.000000   
        com.acobot.en                                             0.000000   
        com.adobe.reader                                          0.000000   
        com.alibaba.aliexpresshd                                  0.000000   
        com.android.calendar                                      0.000000   
        com.android.chrome                                        0.000000   
        com.android.email                                         0.000000   
        com.android.incallui                                      0.000000   
        com.android.phone                                         0.000000   
        com.campursari.photoeditan                                0.000000   
        com.chaisanamobile.kitchendesignideas                    33.333333   
        com.dekovir.pixwords                                      0.000000   
        com.dreamob.android.enphv                                 0.000000   
        com.ebay.mobile                                           0.000000   
        com.facebook.katana                                       0.000000   
        com.facebook.orca                                         0.000000   
        com.fotoable.faceswap.c403                                0.000000   
        com.fotoable.fotobeauty                                   0.000000   
        com.fotoable.photocollage                                 0.000000   
        com.funbox.englishidioms                                  0.000000   
        com.google.android.apps.docs                              0.000000   
        com.google.android.apps.maps                              0.000000   
        com.google.android.apps.photos                            0.000000   
        com.google.android.apps.translate                         0.000000   
        com.google.android.apps.youtube.kids                      0.000000   
        com.google.android.gm                                     0.000000   
        com.google.android.gms                                    0.000000   
        com.google.android.googlequicksearchbox                   0.000000   
        com.google.android.youtube                                0.000000   
        com.google.earth                                          0.000000   
        com.gumtree.android                                       0.000000   
        com.hutchison3g.planet3                                   0.000000   
        com.instagram.android                                     0.000000   
        com.instagram.layout                                      0.000000   
        com.intsig.camscanner                                     0.000000   
        com.jb.zcamera                                            0.000000   
        com.memrise.android.memrisecompanion                      0.000000   
        com.microsoft.office.outlook                              0.000000   
        com.mobisystems.office                                    0.000000   
        com.msvdevelopment.enbulgar.free                          0.000000   
        com.pinterest                                             0.000000   
        com.ryanair.cheapflights                                  0.000000   
        com.sirmamobile.prodavalnik                               0.000000   
        com.skype.raider                                          0.000000   
        com.sonyericsson.album                                    0.000000   
        com.sonyericsson.android.socialphonebook                  0.000000   
        com.sonyericsson.conversations                            0.000000   
        com.sonyericss